# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [22]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2

In [21]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color
                   # channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

This image is: <class 'numpy.ndarray'> with dimesions: (540, 960, 3)


**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

I've added some functionality to the draw_lines function. After I get the points using hough transform, I use them to compute the intercept and slope of the various lines. I used RANSAC which is implemented in the section after this to know which set of slopes and intercepts have the most votes i.e. inlier detection. I use this to then figure out the slope and intercept of the line to be overlayed on the image. If the number of points found by the hough transform is less than four i.e. a bad frame, then the draw_lines function automatically selects the previous slope and intercepts of the previous frame to overlay onto the image.

In [23]:
import random
import math
import sys

#windowSize = 10
prev_m1 = 0
prev_c1 = 0
prev_m2 = 0
prev_c2 = 0

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    mask = cv2.fillPoly(mask, [vertices], ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    global prev_m1
    global prev_m2
    global prev_c1
    global prev_c2
    slopes = []
    i = 0
    
    
    #lines = [[[199, 535, 355, 413]], [[175, 539, 414, 361]], [[743, 467, 862, 538]], [[769, 474, 806, 495]], [[290, 463, 462, 329]],
    #         [[757, 466, 884, 539]], [[582, 370, 602, 382]], [[418, 360, 477, 313]], [[487, 315, 533, 340]], [[507, 324, 533, 338]],
    #         [[545, 348, 568, 360]], [[585, 368, 607, 381]], [[175, 538, 374, 390]], [[454, 331, 468, 320]], [[424, 360, 459, 332]],
    #         [[870, 530, 884, 538]], [[518, 334, 531, 340]], [[547, 346, 565, 357]], [[188, 490, 198, 484]], [[201, 534, 243, 502]],
    #         [[389, 381, 420, 357]], [[264, 485, 282, 484]], [[832, 510, 868, 530]], [[259, 489, 282, 488]], [[743, 466, 865, 539]],
    #         [[503, 324, 530, 340]], [[424, 356, 457, 330]], [[585, 373, 600, 382]], [[584, 368, 606, 380]], [[595, 375, 607, 382]],
    #         [[417, 365, 437, 349]], [[436, 347, 448, 338]], [[510, 326, 533, 339]], [[477, 316, 489, 315]], [[229, 512, 287, 467]]]
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            m = (y2-y1)/(x2-x1)
            c = y2 - m*x2
            m = math.degrees(math.atan(m))
            slopes.append([m,c,[x1,y1],[x2,y2]])
            #cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            
    slopes = sorted(slopes)
    
    #print("length of slopes = ",len(slopes))
    
    breakpoints = []
    f = 1
    avgslope = 0
    avgintercept = 0
    prevBreakpoint = 0
    for i in range(len(slopes)):
        if i > 0:
            if (slopes[i][0]-slopes[i-1][0])>5:
                breakpoints.append([prevBreakpoint, i-1, avgslope/f, avgintercept/f, f])
                prevBreakpoint = i
                avgslope = slopes[i][0]
                avgintercept = slopes[i][1]
                f = 1
            
            elif i == len(slopes)-1:
                f = f + 1
                avgslope = (slopes[i][0]+avgslope)/f
                avgintercept = (slopes[i][1]+avgintercept)/f
                breakpoints.append([prevBreakpoint, i, avgslope, avgintercept, f])
                
            else:
                avgslope = avgslope + slopes[i-1][0]
                avgintercept = avgintercept + slopes[i-1][1]
                f = f + 1
        else:
            avgslope = avgslope + slopes[i-1][0]
            avgintercept = avgintercept + slopes[i-1][1]
    
    breakpoints = sorted(breakpoints, key=lambda breakpoints: breakpoints[4], reverse=True)
    #print(breakpoints)
    
    ##########################################################################################
                                        #FOR RANSAC
    points1 = []
    points2 = []
    
    for i in range(breakpoints[0][0], breakpoints[0][1]):
        points1.append(slopes[i][2])
        points1.append(slopes[i][3])
        
    for i in range(breakpoints[1][0], breakpoints[1][1]):
        points2.append(slopes[i][2])
        points2.append(slopes[i][3])
        
    #if len(breakpoints)<2:
    #    #do nothing
    #    lalala = 0
    #else:
    #    for i in range(breakpoints[0][0], breakpoints[0][1]):
    #        points1.append(slopes[i][2])
    #        points1.append(slopes[i][3])
    #    for i in range(breakpoints[1][0], breakpoints[1][1]):
    #        points2.append(slopes[i][2])
    #        points2.append(slopes[i][3])
        
    if len(points1)<2:
        model_m1 = prev_m1
        model_c1 = prev_c1
    else:
        [model_m1, model_c1] = ransac_points(points1)
    
    if len(points2) < 2:
        model_m2 = prev_m2
        model_c2 = prev_c2
    else:
        [model_m2, model_c2] = ransac_points(points2)
    
    ##########################################################################################
    
    xmax = img.shape[1]#960
    ymax = img.shape[0]#540
    
    #theta1 = breakpoints[0][2]
    #theta2 = breakpoints[1][2]
    #theta1 = math.radians(theta1)
    #theta2 = math.radians(theta2)
        
    #m1 = math.tan(theta1)
    #m2 = math.tan(theta2)
    #c1 = breakpoints[0][3]
    #c2 = breakpoints[1][3]
    
    m1 = model_m1
    m2 = model_m2
    c1 = model_c1
    c2 = model_c2
       
    prev_m1 = m1
    prev_m2 = m2
    prev_c1 = c1
    prev_c2 = c2
    
    if m1 == 0:
        m1 = 0.00001
    if m2 == 0:
        m2 = 0.00001
    
    x = (-1*(c2-c1)/(m2-m1))
    y = (m1*x)+c1
    
    ############################################################################
                            #GETTING POINTS TO DRAW
    
    x1 = ((ymax-c1)/m1)
    y1 = ymax
    x2 = ((ymax-c2)/m2)
    y2 = ymax
    
    #if ((0<=x<=xmax)and(0<=y<=ymax)):
    #    #Get other two points
    #    if(0<=((ymax-c1)/m1)<=xmax):
    #        x1 = ((ymax-c1)/m1)
    #        y1 = ymax
    #    if(0<=((ymax-c2)/m2)<=xmax):
    #        x2 = ((ymax-c2)/m2)
    #        y2 = ymax
    
    x = int(x)
    y = int(y)
    x1 = int(x1)
    y1 = int(y1)
    x2 = int(((ymax-c2)/m2))
    y2 = int(ymax)
    
    width = 40
    x3 = x - width/2
    x4 = x + width/2
    
    if x3<x4 and x1<x2 :
        y3 = int(m1*x3 + c1)
        y4 = int(m2*x4 + c2)
        x3 = int(x3)
        x4 = int(x4)
        cv2.line(img, (x3, y3), (x1, y1), color, thickness)
        cv2.line(img, (x4, y4), (x2, y2), color, thickness)
    else:
        y3 = int(m2*x3 + c2)
        y4 = int(m1*x4 + c1)
        x3 = int(x3)
        x4 = int(x4)
        cv2.line(img, (x4, y4), (x1, y1), color, thickness)
        cv2.line(img, (x3, y3), (x2, y2), color, thickness)
    #print("x = ",x," y = ",y," x1 = ",x1," y1 = ",y1," x2 = ",x2," y2 = ",y2)

## RANSAC Implementation

In [24]:
def find_line_model(points):
    """ find a line model for the given points
    :param points selected points for model fitting
    :return line model
    """
 
    # [WARNING] vertical and horizontal lines should be treated differently
    #           here we just add some noise to avoid division by zero
 
    # find a line model for these points
    m = (points[1][1] - points[0][1]) / (points[1][0] - points[0][0] + sys.float_info.epsilon)  # slope (gradient) of the line
    c = points[1][1] - m * points[1][0]                                     # y-intercept of the line
 
    return m, c

def find_intercept_point(m, c, x0, y0):
    """ find an intercept point of the line model with
        a normal from point (x0,y0) to it
    :param m slope of the line model
    :param c y-intercept of the line model
    :param x0 point's x coordinate
    :param y0 point's y coordinate
    :return intercept point
    """
 
    # intersection point with the model
    x = (x0 + m*y0 - m*c)/(1 + m**2)
    y = (m*x0 + (m**2)*y0 - (m**2)*c)/(1 + m**2) + c
 
    return x, y
    
def ransac_points(points):
    # Ransac parameters
    ransac_iterations = 20  # number of iterations
    ransac_threshold = 3    # threshold
    ransac_ratio = 0.6      # ratio of inliers required to assert
                            # that a model fits well to data
    ratio = 0.
    model_m = 0.
    model_c = 0.
    
    n_samples = len(points) # number of input points
    
    if n_samples < 2:
        print('number of points entered was below 2: no = ', n_samples)
        
    #data = points #np.hstack((x,y))
    
    # perform RANSAC iterations
    for it in range(ransac_iterations):
        
        # pick up two random points
        n = 2
        
        all_indices = list(range(len(points)))
        random.shuffle(all_indices)
        indices_1 = all_indices[:n]
        indices_2 = all_indices[n:]
        
        test_points = []
        #print(indices_1)
        maybe_points = [points[indices_1[0]], points[indices_1[1]]]   #[points[indices_1[0]][:], points[indices_1[1]][:]]
        for i in indices_2:
            test_points.append(points[i])
        
        # find a line model for these points
        m, c = find_line_model(maybe_points)
 
        x_list = []
        y_list = []
        num = 0
 
        # find orthogonal lines to the model for all testing points
        for ind in range(len(test_points)):
 
            x0 = test_points[ind][0]
            y0 = test_points[ind][1]
 
            # find an intercept point of the model with a normal from point (x0,y0)
            x1, y1 = find_intercept_point(m, c, x0, y0)
 
            # distance from point to the model
            dist = math.sqrt((x1 - x0)**2 + (y1 - y0)**2)
 
            # check whether it's an inlier or not
            if dist < ransac_threshold:
                x_list.append(x0)
                y_list.append(y0)
                num += 1
 
        x_inliers = np.array(x_list)
        y_inliers = np.array(y_list)
        
        # in case a new model is better - cache it
        if num/float(n_samples) > ratio:
            ratio = num/float(n_samples)
            model_m = m
            model_c = c
 
        # print ('  inlier ratio = ', num/float(n_samples))
        # print ('  model_m = ', model_m)
        # print ('  model_c = ', model_c)
        # we are done in case we have enough inliers
        if num > n_samples*ransac_ratio:
            break
            #print ('The model is found !')
            #it still doesn't break here!!!??? Whyyyy???
            
    return [model_m, model_c]

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [25]:
import os
os.listdir("test_images/")

['solidWhiteCurve.jpg',
 'solidWhiteRight.jpg',
 'solidYellowCurve.jpg',
 'solidYellowCurve2.jpg',
 'solidYellowLeft.jpg',
 'whiteCarLaneSwitch.jpg']

run your solution on all test_images and make copies into the test_images directory).

In [26]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.

#reading in an image
image_original = mpimg.imread('test_images/whiteCarLaneSwitch.jpg')
image = grayscale(image_original)

#printing out some stats and plotting
#print('This image is:', type(image), 'with dimesions:', image.shape)
#plt.imshow(image, cmap='gray')  #plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

#Apply Canny Edge Detection
low_threshold = 200
high_threshold = 255
image_canny = canny(image, low_threshold, high_threshold)
#plt.imshow(image_canny, cmap='gray')

#Get a smaller area to process
#vertices = np.array([[0, image.shape[1]-1], [image.shape[2]-1, image.shape[1]-1], [((image.shape[2]-1)/2), ((image.shape[1]-1)/2 - 10)]])
ymax = image.shape[0]
xmax = image.shape[1]

xwidth = 10
yoffset = 80

vertices = np.array([[0, ymax],[xmax, ymax],[(xmax + xwidth)/2, (ymax + yoffset)/2],[(xmax - xwidth)/2, (ymax + yoffset)/2]])
vertices = np.int32(vertices)
image_region = region_of_interest(image_canny, vertices)
#plt.imshow(image_region, cmap='gray')

#Apply Hough Transform
rho = 1
theta = 0.01
threshold = 10
min_line_len = 10
max_line_gap = 10
image_hough = hough_lines(image_region, rho, theta, threshold, min_line_len, max_line_gap)

image_processed = weighted_img(image_hough, image_original, α=0.8, β=1., λ=0.)

#plt.imshow(image_hough)
plt.imshow(image_processed)






## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [27]:
# Import everything needed to edit/save/watch video clips

from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [28]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    prev_m1 = 0
    prev_m2 = 0
    prev_c1 = 0
    prev_c2 = 0
    
    image_original = image
    image = grayscale(image)

    #printing out some stats and plotting
    #print('This image is:', type(image), 'with dimesions:', image.shape)
    #plt.imshow(image, cmap='gray')  #plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

    #Apply Canny Edge Detection
    low_threshold = 200
    high_threshold = 255
    image_canny = canny(image, low_threshold, high_threshold)
    #plt.imshow(image_canny, cmap='gray')

    #Get a smaller area to process
    #vertices = np.array([[0, image.shape[1]-1], [image.shape[2]-1, image.shape[1]-1], [((image.shape[2]-1)/2), ((image.shape[1]-1)/2 - 10)]])
    ymax = image.shape[0]
    xmax = image.shape[1]

    xwidth = 10
    yoffset = 80
    loweryCut = int(0.1*ymax)
    lowerxCut = int(0.1*xmax)

    vertices = np.array([[lowerxCut, ymax-loweryCut],[xmax-lowerxCut, ymax-loweryCut],[(xmax + xwidth)/2, (ymax + yoffset)/2],[(xmax - xwidth)/2, (ymax + yoffset)/2]])
    vertices = np.int32(vertices)
    image_region = region_of_interest(image_canny, vertices)
    #plt.imshow(image_region, cmap='gray')

    #Apply Hough Transform
    rho = 1
    theta = 0.01
    threshold = 10
    min_line_len = 10
    max_line_gap = 10
    image_hough = hough_lines(image_region, rho, theta, threshold, min_line_len, max_line_gap)
    
    image_processed = weighted_img(image_hough, image_original, α=0.8, β=1., λ=0.)
    result = image_processed
    return result

Let's try the one with the solid white lane on the right first ...

In [9]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|███████████████████████████████████████████████████████████████▋| 221/222 [00:04<00:00, 50.03it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

Wall time: 4.93 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [10]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [11]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|███████████████████████████████████████████████████████████████▉| 681/682 [00:13<00:00, 60.63it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

Wall time: 13.4 s


In [12]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!

I strongly believe RANSAC is a good method to be used in this scenario because it detects the inliers. However, as my yellow_output video shows, there are still some improper detections. I believe I can remove these by using a moving average filter on the slopes and intercepts for solving it on a basic level. However, an even better option would be to implement a kalman estimator for detecting where the lane should be next, if we observe a significant difference in the estimate and the frame then we can conclude that the frame is bad and take the value of the estimate to overlay the lanes on the image.
I tried my current code without the filter or estimator and it DOES NOT give good results. However, I strongly believe that the estimator will eliminate the lines from the shadows and the sides of the video and give a nice and smooth result.


## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [13]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


100%|████████████████████████████████████████████████████████████████| 251/251 [00:11<00:00, 21.56it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

Wall time: 12.5 s


In [14]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))